# Chapter 7: Skinned Shape Deformation
[![PyPI version](https://badge.fury.io/py/libigl.svg)](https://pypi.org/project/libigl/)
[![buildwheels](https://github.com/libigl/libigl-python-bindings/actions/workflows/wheels.yml/badge.svg)](https://github.com/libigl/libigl-python-bindings/actions/workflows/wheels.yml?query=branch%3Amain)

# Overview

In computer animation, shape deformation is often referred to as “skinning”. Constraints are posed as relative rotations of internal rigid “bones” inside a character. The deformation method, or skinning method, determines how the surface of the character (i.e. its skin) should move as a function of the bone rotations.

In this chapter, we show 3 techniques
1. Rigid Skinning - the most basic shape deformation technique
2. Linear Blend Skinning (LBS) - the most commonly used shape deformation technique
3. Direct Delta Mush (DDM) - one of many advanced deformation techniques to address limitations of LBS

In [1]:
import igl
import scipy as sp
import numpy as np
import meshplot as mp

import os

root_folder = os.getcwd()

# Mesh Loading

Load the elephant mesh along with it's skeleton structure, the weights necessary to control the mesh using the skeleton, and an animation sequence.

In [2]:
v, f = igl.read_triangle_mesh(os.path.join(root_folder, "data", "elephant.obj"))

bones, parents, _, _, _, _ = igl.read_tgf(os.path.join(root_folder, "data", "elephant.tgf"))

w = igl.read_dmat(os.path.join(root_folder, "data", "elephant-weights.dmat"))

anim = igl.read_dmat(os.path.join(root_folder, "data", "elephant-anim.dmat"))

num_frames = anim.shape[1]

print(f"Array Shapes\nVertices: {v.shape}, Faces: {f.shape}, Bones: {bones.shape}, Parents: {parents.shape}, Weights: {w.shape}, Anim: {anim.shape}")

Array Shapes
Vertices: (6034, 3), Faces: (12064, 3), Bones: (25, 3), Parents: (24, 2), Weights: (6034, 24), Anim: (288, 457)


First line loads the elephant mesh which contains the vertices and faces as demonstrated in [chapter 0](https://github.com/libigl/libigl-python-bindings/blob/master/tutorial/tut-chapter0.ipynb)

Second line loads a set of bones and a skeleton heirarchy described by parents.

Third line loads a set of weights $W$ describing how much each vertex ($i$) will be influenced by the bones loaded above ($w_i$). These weights are meant to be used with Linear Blend Skinning.

Fourth line loads an animation sequence where each column describes a pose $\theta$. $\theta$ is a stack of vectorized affine transforms, one for each bone describing the translation and rotation of the bone. In this elephant object example, there are a total of 24 affectable bones. Hence, each column has $3 * 4 * 24 = 288$ elements where the affine transform for each bone is $3$ rows and $4$ columns.

# Rigid Skinning

In the rigid (or simple) skinning approach, each vertex in the mesh is attached to exactly one bone in the skeleton. When the skeleton is posed, the vertices are transformed by their joint’s world space matrix. Every vertex $i$ is transformed by exactly one matrix using the equation $u_i = v_i . W$, where $W$ is the skinning weights matrix loaded earlier.

In [3]:
def convert_lbs_weights_rigid_weights(lbs_w):
    rigid_w = np.zeros(w.shape)
    for i in range(0, w.shape[0]):
        maxj = w[i].argmax()
        for j in range(0, w.shape[1]):
            rigid_w[i, j] = float(maxj == j)
    return rigid_w
    
rigid_w = convert_lbs_weights_rigid_weights(w)
        
rigid_lbs_matrix = igl.lbs_matrix(v, rigid_w)
        
print(f"Example of Conversion at vertex 10\nLBS Weights: {w[10, 5:9]}\nRigid Weights: {rigid_w[10, 5:9]}")

Example of Conversion at vertex 10
LBS Weights: [9.99997436e-01 0.00000000e+00 2.40198315e-06 0.00000000e+00]
Rigid Weights: [1. 0. 0. 0.]


The function `convert_lbs_weights_rigid_weights` is just a helper function to help us convert the LBS weights we loaded into rigid skinning weights. For rigid skinning, each vertex can be inluenced by one bone. So all we do is find the maximum influence for each vertex and set the weight there to 1 and the rest to 0.

The output shows an example of how bones $5, 6, 7$ and $8$ affect vertex $10$ through LBS and Rigid skinning. A value of $1.0$ implies that the respective bone fully controls the vertex and a value of $0.0$ implies that the bone has no inluence on the vertex.

Finally, we use `igl.lbs_matrix` function to produce matrix $M$ as described in the [Fast Automatic Skinning Transformations tutorial](https://libigl.github.io/tutorial/#fast-automatic-skinning-transformations). 

In [4]:
def rigid_deform_mesh(pose):
    rigid_v = rigid_lbs_matrix @ pose
    return rigid_v

viewer_rigid = mp.Viewer({})
rigid = viewer_rigid.add_mesh(v, f, np.array([0.0, 0.5, 0.0]))

@mp.interact(frame=(1, num_frames))
def update_frame(frame):
    frame = frame - 1
    pose = anim[:, frame].reshape(parents.shape[0] * 4, 3, order='F')
    deformed_mesh = rigid_deform_mesh(pose)
    
    viewer_rigid.update_object(oid=rigid, vertices=deformed_mesh)
    
viewer_rigid._renderer

interactive(children=(IntSlider(value=229, description='frame', max=457, min=1), Output()), _dom_classes=('wid…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.1529960…

The function `rigid_deform_mesh` just multiplies the matrix $M$ with the pose $\theta$ to produce the deformed mesh.

# Linear Blend Skinning

In the linear blend skinning approach, each vertex in the mesh can be affected by one or more bones in the skeleton. When the skeleton is posed, the vertices are transformed by doing a weighted sum of joints' world space matrices. The influence of a bone on a vertex can be weighted between $0.0$ and $1.0$ and the sum of influences of bones on each vertex should sum to 1. That is, $\sum_{j=1}^{J} w_i = 1.0,  \forall i \in V$ where $V$ is the set of vertices and $J$ is the number of bones.

In [5]:
lbs_matrix = igl.lbs_matrix(v, w)

def lbs_deform_mesh(pose):
    lbs_v = lbs_matrix @ pose
    return lbs_v

viewer_lbs = mp.Viewer({})
lbs = viewer_lbs.add_mesh(v, f, np.array([0.5, 0.0, 0.0]))

@mp.interact(frame=(1, num_frames))
def update_frame(frame):
    frame = frame - 1
    pose = anim[:, frame].reshape(parents.shape[0] * 4, 3, order='F')
    deformed_mesh = lbs_deform_mesh(pose)
    
    viewer_lbs.update_object(oid=lbs, vertices=deformed_mesh)
    
viewer_lbs._renderer

interactive(children=(IntSlider(value=229, description='frame', max=457, min=1), Output()), _dom_classes=('wid…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.1529960…

The function `lbs_deform_mesh` just multiplies the matrix $M$ with the pose $\theta$ to produce the deformed mesh similar to `rigid_deform_mesh`. The key difference being that `lbs_deform_mesh` uses the matrix $M$ produced by the originally loaded LBS weights $W$.

# Direct Delta Mush Skinning

Linear blend skinning suffers from shrinkage and collapse artifacts due to its inherent linearity. Direct Delta Mush skinning attempts to solve both of these issues by providing a direct skinning method that takes as input a rig with piecewise-constant weight functions (weights are either $=0$ or $=1$ everywhere, i.e weights used for rigid skinning above). Direct delta mush is an adaptation of a less performant method called simply **Delta Mush**. The computation of Delta Mush separates into **“bind pose” precomputation** and **runtime evaluation**.

## "Bind Pose" Precomputation

At bind time, Laplacian smoothing is conducted on the bind pose, moving each vertex from its rest position $v_i$ to a new position $\tilde{v_i}$. The “delta” describing undoing this smoothing procedure, is computed and stored in a local coordinate frame associated with the vertex:

$\delta_i = T_i^{−1}(v_i − \tilde{v_i})$

The result is “vector-valued” skinning weights per-vertex per-bone. This can be stored in a matrix $\Omega$ (i.e `omega`).

In [6]:
p = 20
l = 3
k = 1
a = 0.8
omega = igl.direct_delta_mush_precomputation(v, f, rigid_w, p, l, k, a)

print(f"Num Frames: {num_frames}, Omega: {omega.shape}")

Num Frames: 457, Omega: (6034, 240)


The `igl.direct_delta_mush_precomputation` function generates $\Omega$ that has 10 skinning weights per vertex per bone provided the rest pose vertices, faces, piecewise-constant weights (here we use rigid skinning weight `rigid_w`) and a set of smoothness control parameters. For this example that is 10 weights for each of the 24 bones of the elephant's skeleton for each of 6034 vertices of the mesh.

The smoothness can be controlled through parameters 
- `p` ($ > 0$)
- `l` or $\lambda$ ($> 0$) : 
-`k` or $\kappa$ ($> 0 and < \lambda$):
- 'a' or $\alpha$ ($> 0 and < 1$):

Here, `p` is the number of iterations. The values here were used from [Example 408](https://github.com/libigl/libigl/blob/main/tutorial/408_DirectDeltaMush/main.cpp) of the libigl C++ tutorials.


## Direct Delta Mush Runtime Evaluation

At runtime, $\Omega$ is used to deform the mesh to its final locations. The mesh is deformed using linear blend skinning and piecewise-constant weights. Near bones, the deformation is perfectly rigid, while near joints where bones meet, the mesh tears apart with a sudden change to the next rigid transformation. A local frame $S_i$ is computed at this location and the cached deltas are added in this resolved frame to restore the shape’s original details:

$u_i = \tilde{u_i} + S_i . \delta_i$

The key insight of “Delta Mush” is that Laplacian smoothing acts similarly on the rest and posed models.

In [7]:
def ddm_deform_mesh(pose):
    ddm_v = igl.direct_delta_mush(v.copy(), pose, omega)
    return ddm_v

viewer_ddm = mp.Viewer({})
ddm = viewer_ddm.add_mesh(v, f, np.array([0.0, 0.5, 0.5]))

@mp.interact(frame=(1, num_frames))
def update_frame(frame):
    frame = frame - 1
    pose = anim[:, frame].reshape(parents.shape[0] * 4, 3, order='F')
    deformed_mesh = ddm_deform_mesh(pose)
    
    viewer_ddm.update_object(oid=ddm, vertices=deformed_mesh)
    
viewer_ddm._renderer

interactive(children=(IntSlider(value=229, description='frame', max=457, min=1), Output()), _dom_classes=('wid…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.1529960…

The `igl.direct_delta_mush` consumes the rest pose vertices `v`, the pose $\theta$, and the matrix $\Omega$ we precomputed above to deform the mesh. 

# Comparison between the Skinning Techniques

In [8]:
viewer_comp = mp.Viewer({})
# Rigid Blue
offset_rigid = np.array([-90.0, 0.0, 0.0])
rigid = viewer_comp.add_mesh(v + offset_rigid, f, np.array([0.0, 0.5, 0.0]))

# LBS Red
offset_lbs = np.array([0.0, 0.0, 0.0])
lbs = viewer_comp.add_mesh(v + offset_lbs, f, np.array([0.5, 0.0, 0.0]))

# DDM Green
offset_ddm = np.array([90.0, 0.0, 0.0])
ddm = viewer_comp.add_mesh(v + offset_ddm, f, np.array([0.0, 0.5, 0.5]))


@mp.interact(frame=(1, num_frames))
def update_frame(frame):
    frame = frame - 1
    pose = anim[:, frame].reshape(parents.shape[0] * 4, 3, order='F')
    rigid_deformed_mesh = rigid_deform_mesh(pose)
    lbs_deformed_mesh = lbs_deform_mesh(pose)
    ddm_deformed_mesh = ddm_deform_mesh(pose)
    
    viewer_comp.update_object(oid=rigid, vertices=rigid_deformed_mesh + offset_rigid)
    viewer_comp.update_object(oid=lbs, vertices=lbs_deformed_mesh + offset_lbs)
    viewer_comp.update_object(oid=ddm, vertices=ddm_deformed_mesh + offset_ddm)
    
viewer_comp._renderer

interactive(children=(IntSlider(value=229, description='frame', max=457, min=1), Output()), _dom_classes=('wid…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.1529998…

The visualization shows a comparison between the 3 techniques
1. Rigid Skinning: Green.
2. Linear Blend Skinning: Red.
3. Direct Delta Mush Skinning: Teal.

Frame 181 presents a good case where rigid skinning fails with a ton of artificats, where as linear blend skinning does slightly better. LBS, however, ends up with volume loss at the limbs and Direct Delta Mush skinning does a good job of cleaning this up.

# References

- *UCSD Reading (https://cseweb.ucsd.edu/classes/sp16/cse169-a/readings/3-Skin.html)*
- *Binh Huy Le, J.P. Lewis. [Direct delta mush skinning and variants](https://binh.graphics/papers/2019s-DDM/Direct_Delta_Mush_and_Variants.pdf), 2019*
- *Joe Mancewicz, Matt L. Derksen, Hans Rijpkema, and Cyrus A. Wilson. [Delta Mush: smoothing deformations while preserving detail](https://dl.acm.org/doi/10.1145/2633374.2633376), 2014.*
- *Alec Jacobson, Ilya Baran, Ladislav Kavan, Jovan Popović, and Olga Sorkine. [Fast Automatic Skinning Transformations](https://igl.ethz.ch/projects/fast/fast-automatic-skinning-transformations-siggraph-2012-jacobson-et-al.pdf), 2012*